In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pickle
import glob
from tqdm.notebook import trange, tqdm
import json
import re
import pandas as pd
import numpy as np
from collections import defaultdict
from features import merge_entries
from sklearn.preprocessing import StandardScaler
from features import merge_entries, get_transformer


from utills import cartesian_product, chunker
from train_utils import  generate_doc_pairs_no_chunking, fit_transformers_no_chunking, vectorize_no_chunking


from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc

In [3]:
import matplotlib.pyplot as plt
import matplotlib.style as style
import tikzplotlib
%matplotlib inline

In [4]:

from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [5]:
BASE_PATH = '../data/reddit/'
COMPUTED_DATA_PATH = '../temp_data/reddit/preprocessed/'
TEMP_DATA = '../temp_data/reddit/unchunked_extrmely_limitted_data_40_capped/'
FIGURES_PATH = '../figures/'

In [6]:
MAX_COMMENTS = 40

In [7]:
with open(COMPUTED_DATA_PATH + 'metadata.p', 'rb') as f:
    (train_files, test_files, min_count, author_mapping_all) = pickle.load(f)

In [9]:
1000 / len(train_users)

0.012005522540368569

Vectorized Data
===

In [8]:
train_users = []
with open(COMPUTED_DATA_PATH + 'train.jsonl', 'r') as f:
    for l in tqdm(f):
        d = json.loads(l)
        train_users.append(d['username'])

In [10]:
test_users = []
with open(COMPUTED_DATA_PATH + 'test.jsonl', 'r') as f:
    for l in tqdm(f):
        d = json.loads(l)
        test_users.append(d['username'])

In [11]:
train_user_roots = set()
for y in train_users:
    u = re.search(r'(.*)_[A-Z]+$', y).group(1)
    train_user_roots.add(u)
    
test_user_roots = set()
for y in test_users:
    u = re.search(r'(.*)_[A-Z]+$', y).group(1)
    test_user_roots.add(u)

In [17]:
exclude_users_root = np.random.choice(list(train_user_roots), size=int(len(train_user_roots)*0.96), replace=False)

exclude_users = []
for u in exclude_users_root:
    exclude_users.extend(author_mapping_all[u])

In [19]:
exclude_users_root_test = np.random.choice(list(test_user_roots), size=int(len(test_user_roots)*0.96), replace=False)

exclude_users_test = []
for u in exclude_users_root_test:
    exclude_users_test.extend(author_mapping_all[u])

In [20]:
transformer, scaler, secondary_scaler = fit_transformers_no_chunking(COMPUTED_DATA_PATH + 'train.jsonl', author_mapping_all, max_comments=MAX_COMMENTS, exclude_users=exclude_users)

Sampled: 1087
Reading preprocessed data...



Fitting transformer
Generating pairs
new2



143 147 17
143 42 17


In [21]:
XX_train, author_to_doc_idx, author_subreddit, x_shape, e, t = vectorize_no_chunking(
    preprocessed_path = COMPUTED_DATA_PATH + 'train.jsonl', 
    vectorized_x_path = TEMP_DATA + 'XX_train.npy', 
    transformer=transformer,
    scaler=scaler,
    max_comments=MAX_COMMENTS,
    exclude_users=exclude_users
)

Precomputing record size...


In [22]:
print('Excluded: ', len(e), 'Total:', t)

Excluded:  79967 Total: 83295


In [23]:
len(author_subreddit), len(exclude_users)

(3328, 80067)

In [24]:
XX_test, author_to_doc_idx_test, author_subreddit_test, x_shape_test, e, t = vectorize_no_chunking(
    preprocessed_path = COMPUTED_DATA_PATH + 'test.jsonl', 
    vectorized_x_path = TEMP_DATA + 'XX_test.npy', 
    transformer=transformer,
    scaler=scaler,
    max_comments=MAX_COMMENTS,
    exclude_users=exclude_users_test
)

Precomputing record size...


In [25]:
print('Excluded: ', len(e), 'Total:', t)

Excluded:  41238 Total: 42961


In [26]:
len(author_subreddit_test)

1723

In [27]:
author_mapping = defaultdict(set)
author_to_root = {}
for y in author_to_doc_idx.keys():
    u = re.search(r'(.*)_[A-Z]+$', y).group(1)
    author_mapping[u].add(y)
    author_to_root[y] = u

subreddit_to_author = defaultdict(list)
for k, v in author_subreddit.items():
    subreddit_to_author[v].append(k)
    
author_mapping_test = defaultdict(set)
author_to_root_test = {}
for y in author_to_doc_idx_test.keys():
    u = re.search(r'(.*)_[A-Z]+$', y).group(1)
    author_mapping_test[u].add(y)
    author_to_root_test[y] = u

subreddit_to_author_test = defaultdict(list)
for k, v in author_subreddit_test.items():
    subreddit_to_author_test[v].append(k)

In [28]:
train_authors = np.unique(list(author_to_root.values()))
test_authors = np.unique(list(author_to_root_test.values()))

print('Total number of authors:', len(train_authors) + len(test_authors))
print('Train authors:', len(train_authors))
print('Test authors:', len(test_authors))

Total number of authors: 1860
Train authors: 1220
Test authors: 640


In [29]:
X_idxs_train, Y_train = generate_doc_pairs_no_chunking(author_mapping, subreddit_to_author, author_to_root, author_to_doc_idx, author_subreddit)    
X_idxs_test, Y_test = generate_doc_pairs_no_chunking(author_mapping_test, subreddit_to_author_test, author_to_root_test, author_to_doc_idx_test, author_subreddit_test)

new2



3556 3328 1663
3556 1066 1663
new2



1864 1723 697
1864 559 697


Train the classifier
===

In [30]:
batch_sz = 10000
x_test_diff_sample = secondary_scaler.transform(np.abs(XX_test[X_idxs_test[:batch_sz, 0]] - XX_test[X_idxs_test[:batch_sz, 1]]))
y_test_sample = Y_test[:batch_sz]

In [31]:
len(X_idxs_train)

6285

In [65]:
batch_sz = 10000
clf = SGDClassifier(loss='log', alpha=0.01)
aucs = []
for i in range(100):
    for idxs in chunker(np.arange(len(X_idxs_train)), batch_sz):
        x_diff = secondary_scaler.transform(np.abs(XX_train[X_idxs_train[idxs, 0]] - XX_train[X_idxs_train[idxs, 1]]))
        x_diff[np.isnan(x_diff)]=0
        y = Y_train[idxs]
        clf.partial_fit(x_diff, y, classes=[0, 1])

        probs = clf.predict_proba(x_test_diff_sample)[:, 1]

        fpr, tpr, thresh = roc_curve(y_test_sample, probs)
        roc_auc = auc(fpr, tpr)
        print('AUC:', roc_auc)
    print('~'*20, 'Epoch: ', i)
    aucs.append(roc_auc)

AUC: 0.8739320645144875
~~~~~~~~~~~~~~~~~~~~ Epoch:  0
AUC: 0.8963148473643368
~~~~~~~~~~~~~~~~~~~~ Epoch:  1
AUC: 0.9004108748934583
~~~~~~~~~~~~~~~~~~~~ Epoch:  2
AUC: 0.9036706025281673
~~~~~~~~~~~~~~~~~~~~ Epoch:  3
AUC: 0.90237434431756
~~~~~~~~~~~~~~~~~~~~ Epoch:  4
AUC: 0.9058046182942292
~~~~~~~~~~~~~~~~~~~~ Epoch:  5
AUC: 0.8942599428384845
~~~~~~~~~~~~~~~~~~~~ Epoch:  6
AUC: 0.9027676477697255
~~~~~~~~~~~~~~~~~~~~ Epoch:  7
AUC: 0.9035478939713258
~~~~~~~~~~~~~~~~~~~~ Epoch:  8
AUC: 0.898118239103056
~~~~~~~~~~~~~~~~~~~~ Epoch:  9
AUC: 0.9082430201888704
~~~~~~~~~~~~~~~~~~~~ Epoch:  10
AUC: 0.9088195913884568
~~~~~~~~~~~~~~~~~~~~ Epoch:  11
AUC: 0.9051180599432624
~~~~~~~~~~~~~~~~~~~~ Epoch:  12
AUC: 0.9046835444379895
~~~~~~~~~~~~~~~~~~~~ Epoch:  13
AUC: 0.9008850122761562
~~~~~~~~~~~~~~~~~~~~ Epoch:  14
AUC: 0.9063854299623022
~~~~~~~~~~~~~~~~~~~~ Epoch:  15
AUC: 0.905839867188527
~~~~~~~~~~~~~~~~~~~~ Epoch:  16
AUC: 0.9036087181911857
~~~~~~~~~~~~~~~~~~~~ Epoch:  17
AUC: 0

In [66]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=aucs))


In [23]:
TEMP_DATA

'../temp_data/reddit/unchunked_limitted_data_40_capped/'

In [67]:
with open(TEMP_DATA + 'model.p', 'wb') as f:
    pickle.dump((clf, transformer, scaler, secondary_scaler, aucs ), f)

In [68]:
with open(TEMP_DATA + 'experiment_data.p', 'wb') as f:
    pickle.dump((
        author_to_doc_idx, 
        author_to_doc_idx_test, 
        author_subreddit, 
        author_subreddit_test, 
        x_shape, 
        x_shape_test,
        X_idxs_train,
        Y_train,
        X_idxs_test,
        Y_test,
        exclude_users,
        exclude_users_root,
        exclude_users_test,
        exclude_users_root_test
    ), f)

In [69]:
x_diff = secondary_scaler.transform(np.abs(XX_test[X_idxs_test[:, 0]] - XX_test[X_idxs_test[:, 1]]))
x_diff[np.isnan(x_diff)]=0

probs = clf.predict_proba(x_diff)[:, 1]
fpr, tpr, thresh = roc_curve(Y_test, probs)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

AUC: 0.9128400855726541


In [ ]:
aucs[-1]